In [1]:
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
from pykrx import stock

In [2]:
# setting the path
folderPath = Path.cwd().joinpath('DataStock')
os.chdir(folderPath)

## **Stock Price DB**

### **1. Definition**

#### 1.1. Old Definition (too complicated, too slow! thus, not used!)

In [2]:
# new empty df (if you're trying to make the first df)
def emptyStockPriceDf():
    cols = ['ticker', 'name', 'open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame(columns = cols)
    df.index.name = 'date'
    return df

In [3]:
def updateDf(oldDf, date, ticker) :
    date = date
    df = oldDf
    # for ticker in tickers :
    print(f'{date} : {ticker}')
    newDf = stock.get_market_ohlcv_by_date(date, date, ticker)
    cols = ['open', 'high', 'low', 'close', 'volume']
    newDf.columns = cols
    newDf = newDf.assign(ticker = ticker)
    newDf = newDf.assign(name = stock.get_market_ticker_name(ticker))
    newDf.index.name = 'date'
    df = pd.concat([df, newDf])
    return df

In [4]:
def queryStockListDf(folderPath) : 
    conn = sqlite3.connect(folderPath/'StockListDB.db', isolation_level=None)
    stockListDf = pd.read_sql_query('SELECT * FROM StockListDB', conn)
    stockListDf = stockListDf.assign(key=stockListDf.apply(lambda x: x.loc['ticker']+x.loc['name'], axis=1))
    stockListDf = stockListDf.drop('id', axis=1)
    conn.close()
    stockListDf.firstDay = pd.to_datetime(stockListDf.firstDay, format = '%Y-%m-%d')
    stockListDf.endDay = pd.to_datetime(stockListDf.endDay, format = '%Y-%m-%d')
    return stockListDf

#### 1.2. Current Used Definition

In [5]:
def getDayList(startDay, endDay):
    # Day List
    start = datetime.strptime(startDay, "%Y%m%d")
    end = datetime.strptime(endDay, "%Y%m%d")
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days+1)]

    dayList = []
    for date in date_generated:
        dayList.append(date.strftime("%Y%m%d"))

    return dayList

In [6]:
def updateDfByAllStock(oldDf, date) :
    date = date
    df = oldDf
    newDf = stock.get_market_ohlcv_by_ticker(date, market="KOSPI")
    cols = ['open', 'high', 'low', 'close', 'volume', 'volume($)', 'change']
    newDf.columns = cols
    newDf = newDf.assign(date = date)
    newDf.index.name = 'ticker'
    newDf = newDf.assign(name=newDf.apply(lambda x : stock.get_market_ticker_name(x.name), axis=1))
    newDf = newDf.reset_index()
    if not newDf.close.sum() == 0 :
        df = pd.concat([df, newDf])
    return df

### **2. Creating DB file**

In [21]:
conn = sqlite3.connect('stockPriceDB.db')
cur = conn.cursor()
conn.execute(
    'CREATE TABLE stockPriceDB (id INTEGER PRIMARY KEY AUTOINCREMENT, date TIMESTAMP, ticker TEXT,  name TEXT, open REAL, high REAL, low REAL, close REAL, volume REAL)')
conn.commit()
conn.close()

### **3. Insert data into DB**

#### 3.1. Make the stage df

In [8]:
cols = ['date','ticker', 'open', 'high', 'low', 'close', 'volume', 'name']
stockPriceDB = pd.DataFrame(columns=cols)
stockPriceDB.index.name='ticker'

dayList = getDayList('20100101', '20211008')

for day in dayList :
    print(day)
    stockPriceDB = updateDfByAllStock(stockPriceDB, day)
    stockPriceDB = stockPriceDB[['date','ticker','name', 'open', 'high', 'low', 'close', 'volume']]
stockPriceDB['date'] = pd.to_datetime(stockPriceDB['date'], format = '%Y-%m-%d')
stockPriceDB

20100101
20100102
20100103
20100104
20100105
20100106
20100107
20100108
20100109
20100110
20100111
20100112
20100113
20100114
20100115
20100116
20100117
20100118
20100119
20100120
20100121
20100122
20100123
20100124
20100125
20100126
20100127
20100128
20100129
20100130
20100131
20100201
20100202
20100203
20100204
20100205
20100206
20100207
20100208
20100209
20100210
20100211
20100212
20100213
20100214
20100215
20100216
20100217
20100218
20100219
20100220
20100221
20100222
20100223
20100224
20100225
20100226
20100227
20100228
20100301
20100302
20100303
20100304
20100305
20100306
20100307
20100308
20100309
20100310
20100311
20100312
20100313
20100314
20100315
20100316
20100317
20100318
20100319
20100320
20100321
20100322
20100323
20100324
20100325
20100326
20100327
20100328
20100329
20100330
20100331
20100401
20100402
20100403
20100404
20100405
20100406
20100407
20100408
20100409
20100410
20100411
20100412
20100413
20100414
20100415
20100416
20100417
20100418
20100419
20100420
20100421
2

,date,ticker,name,open,high,low,close,volume
0,2010-01-04,004560,현대비앤지스틸,8660,8910,8650,8910,138442
1,2010-01-04,004565,현대비앤지스틸우,8370,8370,8370,8370,980
2,2010-01-04,001460,BYC,122000,125000,122000,125000,497
3,2010-01-04,001465,BYC우,55000,56300,53800,55400,18
4,2010-01-04,084680,이월드,1115,1135,1080,1110,776027
...,...,...,...,...,...,...,...,...
933,2021-10-08,069260,휴켐스,27450,27550,26650,26800,458956
934,2021-10-08,000540,흥국화재,3730,3745,3640,3655,165721
935,2021-10-08,000547,흥국화재2우B,28900,28900,27750,28800,170
936,2021-10-08,000545,흥국화재우,8500,8630,8370,8590,657


#### 3.2. Ready the Staged Changes

In [28]:
stockPriceDB = stockPriceDB.reset_index(drop=True)
# stockPriceDB['date'] = stockPriceDB['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
for row in stockPriceDB.iloc[:5].itertuples():
    print(row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], sep='/')

2010-01-04/004560/현대비앤지스틸/8660.0/8910.0/8650.0/8910.0/138442.0
2010-01-04/004565/현대비앤지스틸우/8370.0/8370.0/8370.0/8370.0/980.0
2010-01-04/001460/BYC/122000.0/125000.0/122000.0/125000.0/497.0
2010-01-04/001465/BYC우/55000.0/56300.0/53800.0/55400.0/18.0
2010-01-04/084680/이월드/1115.0/1135.0/1080.0/1110.0/776027.0


#### 3.3. Insert data into DB

In [29]:
connect = sqlite3.connect('./stockPriceDB.db')
cursor = connect.cursor()
for row in stockPriceDB.itertuples():
    sql = "insert into stockPriceDB (date, ticker, name, open, high, low, close, volume) values (?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
connect.commit()
connect.close()

#### 3.4. Save DB as the format of Parquet

In [ ]:
stockPriceDB['date'] = pd.to_datetime(stockPriceDB['date'], format = '%Y-%m-%d')
stockPriceDB.to_parquet(folderPath/'stockPriceDB.parquet')

### **4. Query DB**

In [16]:
conn = sqlite3.connect('./stockPriceDB.db', isolation_level=None)
stockPriceDB = pd.read_sql_query('SELECT * FROM stockPriceDB', conn)
stockPriceDB['date'] = pd.to_datetime(stockPriceDB['date'], format='%Y-%m-%d') # str(format 2021-11-27) -> datetime
conn.close()
stockPriceDB

,id,date,ticker,name,open,high,low,close,volume
0,1,2010-01-04,004560,현대비앤지스틸,8660.0,8910.0,8650.0,8910.0,138442.0
1,2,2010-01-04,004565,현대비앤지스틸우,8370.0,8370.0,8370.0,8370.0,980.0
2,3,2010-01-04,001460,BYC,122000.0,125000.0,122000.0,125000.0,497.0
3,4,2010-01-04,001465,BYC우,55000.0,56300.0,53800.0,55400.0,18.0
4,5,2010-01-04,084680,이월드,1115.0,1135.0,1080.0,1110.0,776027.0
...,...,...,...,...,...,...,...,...,...
2640211,2640212,2021-10-08,069260,휴켐스,27450.0,27550.0,26650.0,26800.0,458956.0
2640212,2640213,2021-10-08,000540,흥국화재,3730.0,3745.0,3640.0,3655.0,165721.0
2640213,2640214,2021-10-08,000547,흥국화재2우B,28900.0,28900.0,27750.0,28800.0,170.0
2640214,2640215,2021-10-08,000545,흥국화재우,8500.0,8630.0,8370.0,8590.0,657.0
